# Load dim_order_shipping từ Silver lên Gold

## Mục tiêu
- Load dữ liệu từ Silver.dim_order_shipping → Gold.dim_order_shipping
- Không cần biến đổi dữ liệu
- Chỉ cần copy và load vào Gold database

## Quy trình 3 bước
1. **Load from Silver** - Đọc dữ liệu từ Silver database
2. **Schema Definition** - Định nghĩa schema cho MySQL
3. **Load to Gold** - Load vào Gold database


## Bước 1: Import và Setup


In [1]:
# Import các thư viện cần thiết
import os
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
from datetime import datetime

# Load biến môi trường từ file .env
load_dotenv()

# Lấy thông tin kết nối database
DB_USER = os.getenv("DB_USER")
DB_PASS = os.getenv("DB_PASS")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_SILVER = os.getenv("DB_SILVER")
DB_GOLD = os.getenv("DB_GOLD")

# Tạo kết nối tới Silver và Gold database
silver_engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_SILVER}")
gold_engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_GOLD}")

print("Đã kết nối Silver và Gold database thành công")
print(f"Silver DB: {DB_SILVER}")
print(f"Gold DB: {DB_GOLD}")
print(f"Thời gian bắt đầu: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


Đã kết nối Silver và Gold database thành công
Silver DB: winner_silver
Gold DB: winner_gold
Thời gian bắt đầu: 2025-10-17 23:37:02


## Bước 2: Load dữ liệu từ Silver


In [2]:
# Load dữ liệu dim_order_shipping từ Silver database
print("=== LOADING DIM_ORDER_SHIPPING FROM SILVER ===")

# Load toàn bộ bảng dim_order_shipping
shipping_df = pd.read_sql("SELECT * FROM dim_order_shipping", silver_engine)

# Hiển thị thông tin cơ bản về dataset
print(f"Shape: {shipping_df.shape}")
print(f"Columns: {list(shipping_df.columns)}")
print(f"Memory usage: {shipping_df.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")

# Hiển thị sample data
print("\n=== SAMPLE DATA ===")
print(shipping_df.head(5))

# Hiển thị data types
print("\n=== DATA TYPES ===")
print(shipping_df.dtypes)

# Kiểm tra null values
print(f"\n=== NULL VALUES ===")
null_counts = shipping_df.isnull().sum()
print(null_counts[null_counts > 0])


=== LOADING DIM_ORDER_SHIPPING FROM SILVER ===
Shape: (40236, 7)
Columns: ['shipping_id', 'shipping_full_name', 'shipping_phone', 'shipping_full_address', 'shipping_commune_id', 'shipping_district_id', 'shipping_province_id']
Memory usage: 20.50 MB

=== SAMPLE DATA ===
  shipping_id shipping_full_name shipping_phone  \
0  SHIP_40616          Thinh Bui     0903693389   
1  SHIP_40615        Truong Minh     0907809070   
2  SHIP_40614        Quach Quach     0986533988   
3  SHIP_40613            Dung Vu     0977752936   
4  SHIP_40612         trần khánh     0978158273   

                               shipping_full_address shipping_commune_id  \
0                                               None                None   
1                                               None                None   
2                                               None                None   
3                                               None                None   
4  thôn Thái sơn, Xã Đình Chu, Huyện Lập Th

## Bước 3: Load vào Gold Database


In [3]:
# Load vào Gold Database
print("=== LOADING TO GOLD DATABASE ===")

from sqlalchemy.types import BigInteger, Integer, String, Text, DateTime, Float

# Định nghĩa schema mapping cho MySQL
dtype_mapping = {}

# ID fields - String (để xử lý mixed data types)
id_fields = ["shipping_id", "order_id"]
for field in id_fields:
    if field in shipping_df.columns:
        dtype_mapping[field] = String(100)

# String fields - VARCHAR(255)
string_fields = ["shipping_method", "shipping_status", "carrier_name", "tracking_number"]
for field in string_fields:
    if field in shipping_df.columns:
        dtype_mapping[field] = String(255)

# Text fields - Text()
text_fields = ["shipping_address", "shipping_note"]
for field in text_fields:
    if field in shipping_df.columns:
        dtype_mapping[field] = Text()

# Numeric fields
float_fields = ["shipping_fee", "weight", "distance"]
for field in float_fields:
    if field in shipping_df.columns:
        dtype_mapping[field] = Float()

# DateTime fields
datetime_fields = ["created_at", "updated_at", "shipped_at", "delivered_at", "estimated_delivery"]
for field in datetime_fields:
    if field in shipping_df.columns:
        dtype_mapping[field] = DateTime()

print(f"Schema mapping defined for {len(dtype_mapping)} columns")

# Load vào Gold database
table_name = "dim_order_shipping"

try:
    shipping_df.to_sql(
        table_name,
        con=gold_engine,
        if_exists="replace",  # Ghi đè dữ liệu cũ
        index=False,
        dtype=dtype_mapping
    )
    
    print(f"Đã load {shipping_df.shape[0]} records vào Gold: {table_name}")
    
    # Verify load
    verification_query = f"SELECT COUNT(*) as count FROM {table_name}"
    verification_result = pd.read_sql(verification_query, gold_engine)
    loaded_count = verification_result['count'].iloc[0]
    
    print(f"Verification: {loaded_count} records trong database")
    print(f"Schema: {len(dtype_mapping)} columns với explicit typing")
    
    # Sample data từ database
    sample_query = f"SELECT * FROM {table_name} LIMIT 5"
    sample_db = pd.read_sql(sample_query, gold_engine)
    print(f"\nSample data từ Gold database:\n{sample_db}")
    
except Exception as e:
    print(f"Error loading to Gold: {str(e)}")
    raise

print(f"\nLoad dim_order_shipping completed successfully!")
print(f"Ready for EDA and analytics")


=== LOADING TO GOLD DATABASE ===
Schema mapping defined for 1 columns
Đã load 40236 records vào Gold: dim_order_shipping
Verification: 40236 records trong database
Schema: 1 columns với explicit typing

Sample data từ Gold database:
  shipping_id shipping_full_name shipping_phone  \
0  SHIP_40616          Thinh Bui     0903693389   
1  SHIP_40615        Truong Minh     0907809070   
2  SHIP_40614        Quach Quach     0986533988   
3  SHIP_40613            Dung Vu     0977752936   
4  SHIP_40612         trần khánh     0978158273   

                               shipping_full_address shipping_commune_id  \
0                                               None                None   
1                                               None                None   
2                                               None                None   
3                                               None                None   
4  thôn Thái sơn, Xã Đình Chu, Huyện Lập Thạch, V...             1040371   

  s

## Bước 4: Tạo Data Dictionary cho Gold


In [4]:
# Tạo Data Dictionary cho Gold
print("=== CREATING GOLD DATA DICTIONARY ===")

def get_business_meaning_vietnamese(column_name):
    """Get business meaning for each column in Vietnamese"""
    business_meanings = {
        # Định danh & Cơ bản
        "shipping_id": "Mã định danh vận chuyển duy nhất (khóa chính)",
        "order_id": "Mã định danh đơn hàng (khóa ngoại đến fact_orders)",
        
        # Thông tin vận chuyển
        "shipping_method": "Phương thức vận chuyển (Standard, Express, etc.)",
        "shipping_status": "Trạng thái vận chuyển (pending, shipped, delivered, etc.)",
        "carrier_name": "Tên đơn vị vận chuyển (Viettel Post, Giao Hàng Nhanh, etc.)",
        "tracking_number": "Mã theo dõi đơn hàng",
        "shipping_address": "Địa chỉ giao hàng đầy đủ",
        "shipping_note": "Ghi chú về vận chuyển",
        
        # Chi phí và thông số
        "shipping_fee": "Phí vận chuyển (VND)",
        "weight": "Trọng lượng gói hàng (kg)",
        "distance": "Khoảng cách vận chuyển (km)",
        
        # Thời gian
        "created_at": "Thời gian tạo bản ghi vận chuyển",
        "updated_at": "Thời gian cập nhật bản ghi cuối cùng",
        "shipped_at": "Thời gian bắt đầu vận chuyển",
        "delivered_at": "Thời gian giao hàng thành công",
        "estimated_delivery": "Thời gian dự kiến giao hàng"
    }
    return business_meanings.get(column_name, "Chưa định nghĩa ý nghĩa business")

# Tạo Data Dictionary
dict_data = []
for col in shipping_df.columns:
    col_info = {
        "table_name": "dim_order_shipping",
        "column_name": col,
        "dtype": str(shipping_df[col].dtype),
        "sql_type": str(dtype_mapping.get(col, "Not defined")),
        "null_count": shipping_df[col].isnull().sum(),
        "null_pct": round(shipping_df[col].isnull().mean() * 100, 2),
        "unique_count": shipping_df[col].nunique(),
        "sample_values": str(shipping_df[col].dropna().unique()[:3].tolist()),
        "business_meaning": get_business_meaning_vietnamese(col),
        "extraction_date": datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }
    dict_data.append(col_info)

data_dictionary = pd.DataFrame(dict_data)

# Hiển thị Data Dictionary
print(f"Generated Data Dictionary for {len(data_dictionary)} columns")
print("\n=== DATA DICTIONARY ===")
print(data_dictionary)

# Lưu vào file Excel
excel_path = "Technical_Document/Gold_dictionary.xlsx"

try:
    from openpyxl import load_workbook
    
    # Kiểm tra file Excel có tồn tại không
    try:
        # Load workbook hiện tại
        wb = load_workbook(excel_path)
        
        # Lấy sheet đầu tiên
        ws = wb.active
        
        # Kiểm tra xem có dữ liệu cũ không
        if ws.max_row > 1:
            print(f"Found existing data in {excel_path}, appending new data...")
        else:
            print(f"File {excel_path} exists but is empty, adding header and data...")
            
    except FileNotFoundError:
        print(f"File {excel_path} not found, creating new file...")
        wb = None
    except Exception as e:
        print(f"Error loading {excel_path}: {str(e)}, creating new file...")
        wb = None
    
    if wb is None:
        # Tạo file mới với header
        data_dictionary.to_excel(excel_path, index=False, sheet_name='Gold_Dictionary')
        print(f"✅ Created new file: {excel_path}")
    else:
        # Append vào file hiện tại
        from openpyxl.utils.dataframe import dataframe_to_rows
        
        # Tìm dòng cuối cùng có dữ liệu
        last_row = ws.max_row
        
        # Thêm dữ liệu mới từ dòng tiếp theo
        for r in dataframe_to_rows(data_dictionary, index=False, header=False):
            last_row += 1
            for c_idx, value in enumerate(r, 1):
                ws.cell(row=last_row, column=c_idx, value=value)
        
        # Lưu file
        wb.save(excel_path)
        print(f"✅ Appended {len(data_dictionary)} rows to: {excel_path}")
    
except Exception as e:
    print(f"❌ Error appending to Data Dictionary: {str(e)}")
    # Fallback: tạo file mới
    try:
        data_dictionary.to_excel(excel_path, index=False)
        print(f"✅ Created new file as fallback: {excel_path}")
    except Exception as e2:
        print(f"❌ Error creating fallback file: {str(e2)}")

# Summary Report
print(f"\n=== GOLD DICTIONARY SUMMARY ===")
print(f"Table: dim_order_shipping")
print(f"Total columns: {len(data_dictionary)}")
print(f"Data Dictionary: {excel_path}")
print(f"Created at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

print(f"\n=== SAMPLE DICTIONARY ENTRIES ===")
sample_dict = data_dictionary[['column_name', 'dtype', 'business_meaning']].head(5)
print(sample_dict)


=== CREATING GOLD DATA DICTIONARY ===
Generated Data Dictionary for 7 columns

=== DATA DICTIONARY ===
           table_name            column_name   dtype      sql_type  \
0  dim_order_shipping            shipping_id  object  VARCHAR(100)   
1  dim_order_shipping     shipping_full_name  object   Not defined   
2  dim_order_shipping         shipping_phone  object   Not defined   
3  dim_order_shipping  shipping_full_address  object   Not defined   
4  dim_order_shipping    shipping_commune_id  object   Not defined   
5  dim_order_shipping   shipping_district_id  object   Not defined   
6  dim_order_shipping   shipping_province_id  object   Not defined   

   null_count  null_pct  unique_count  \
0           0      0.00         40236   
1           0      0.00         25326   
2          14      0.03         36100   
3        4657     11.57         32538   
4        4695     11.67          7343   
5        4690     11.66           701   
6        4678     11.63            63   

       